In [ ]:
from typing import Optional, Union, Tuple, List, Callable, Dict
import torch
from diffusers import StableDiffusionPipeline
import torch.nn.functional as nnf
import numpy as np
import abc
import ptp_utils
import os
import torch.nn as nn
from attention_controller import *
import cv2
import matplotlib.pyplot as plt
import diffusers
from PIL import Image, ImageDraw, ImageFont
from skimage import morphology

print("diffusers:",diffusers.__version__)


In [ ]:
MY_TOKEN = ''
LOW_RESOURCE = False
NUM_DIFFUSION_STEPS = 50
GUIDANCE_SCALE = 7.5
MAX_NUM_WORDS = 77
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
your_SD_model_path = ''
ldm_stable = StableDiffusionPipeline.from_pretrained(your_SD_model_path, use_auth_token=MY_TOKEN).to(device)
tokenizer = ldm_stable.tokenizer
your_out_folder_path = ''
output_folder = your_out_folder_path
if not os.path.exists(output_folder):
    os.mkdir(output_folder)


In [ ]:
def run_and_display(prompts, controller, if_process = False, in_img = None, full_img = None,if_save = False, latent=None, run_baseline=False, generator=None,sqr_replace=False,switch = False, bbox = None,img_name = None , output_folder = None, box = None):
    images, x_t = ptp_utils.text2image_ldm_stable(ldm_stable, prompts, controller, real_img=in_img, full_img = full_img, latent=latent, num_inference_steps=NUM_DIFFUSION_STEPS, guidance_scale=GUIDANCE_SCALE, generator=generator, low_resource=LOW_RESOURCE,is_switch=switch,bboxes=bbox)
    ptp_utils.view_images(images,if_save, img_name = img_name, output_folder=output_folder,box=box)
    return images, x_t

In [ ]:
g_cpu = torch.Generator().manual_seed(8888)

In [ ]:
def view_images(images, if_save = False, num_rows=1, offset_ratio=0.02,output_folder = None, save_which = -1, img_name = None, box = None):
    if type(images) is list:
        num_empty = len(images) % num_rows
    elif images.ndim == 4:
        num_empty = images.shape[0] % num_rows
    else:
        images = [images]
        num_empty = 0

    empty_images = np.ones(images[0].shape, dtype=np.uint8) * 255
    images = [image.astype(np.uint8) for image in images] + [empty_images] * num_empty
    num_items = len(images)

    h, w, c = images[0].shape
    offset = int(h * offset_ratio)
    num_cols = num_items // num_rows
    image_ = np.ones((h * num_rows + offset * (num_rows - 1),
                      w * num_cols + offset * (num_cols - 1), 3), dtype=np.uint8) * 255
    for i in range(num_rows):
        for j in range(num_cols):
            image_[i * (h + offset): i * (h + offset) + h:, j * (w + offset): j * (w + offset) + w] = images[
                i * num_cols + j]

    if if_save:
        out_img = images[save_which]
        if box is not None:
            the_box = box[0]
            out_img = Image.fromarray(out_img)
            a = ImageDraw.ImageDraw(out_img)
            for sub_box in the_box:
                print('sub_box: ',sub_box)
                a.rectangle(((sub_box[0], sub_box[1]), (sub_box[2], sub_box[3])), fill=None, outline='blue', width=2)
            out_img.save(output_folder + img_name + " Rec.jpg")
        else:
            im = Image.fromarray(out_img)
            im.save(output_folder + img_name + ".jpg")

## Ours Method

In [ ]:
real_img = None
full_msk = None
real_img_full = None
g_cpu = torch.Generator().manual_seed(8888)

out_path_name = './TPAMI_exp_out'
in_path = './mask_info/'

out_path_list = [out_path_name + "_no_process/",out_path_name + "_edit/",out_path_name + "_ref/"]

for path in out_path_list:
    if not os.path.exists(path):
        os.mkdir(path)

file_list = os.listdir(in_path)
img_namelist = os.listdir(out_path_list[1])

or_file =  open("./prompt_original.txt","r")
lines = or_file.read().splitlines()

for idx, filename in enumerate(lines):

    '''
    if filename + '.jpg' in img_namelist:
        continue
    '''
    prompts = ["","",""]
    with open(in_path + filename + '.txt', "r") as f:
        contents = f.readlines()
    
    x1 = int(contents[0].strip('\n'))
    y1 = int(contents[1].strip('\n'))
    w = int(contents[2].strip('\n'))
    h = int(contents[3].strip('\n'))
    prompts[-1] = contents[-1].strip('\n')

    if 'no' in prompts[-1] or 'No' in prompts[-1]:
        _ = ptp_utils.init_latent(None, ldm_stable, 512, 512, g_cpu, 3)
        continue

    prompts[0] = filename

    tmp_w = w
    w = h
    h = tmp_w

    tmp_x1 = x1
    x1 = y1
    y1 = tmp_x1

    size_re = w
    if w < h:
        size_re = h


    alpha_idx = 0

    if h < w*alpha_idx:
        h = int(w*alpha_idx)
    elif w < h*alpha_idx:
        w = int(h*alpha_idx)

    x2 = x1 + w - 1 
    y2 = y1 + h - 1

    att_mask = np.zeros((512,512))

    for i in range(x1,x2+1):
        for j in range(y1,y2+1):
            att_mask[j,i] = 1

    beta_idx = 0.1

    x1_r = max(x1/att_mask.shape[0],beta_idx)
    y1_r = max(y1/att_mask.shape[1],beta_idx)
    x2_r = max(x2/att_mask.shape[0],beta_idx)
    y2_r = max(y2/att_mask.shape[1],beta_idx)

    box = [[[x1_r,y1_r,x2_r,y2_r]]]

    controller = AttentionReplace(prompts, NUM_DIFFUSION_STEPS, cross_replace_steps=.4, self_replace_steps=0.4, where_change = 1,mask = att_mask, full_mask = full_msk)


    iamges,_ = run_and_display(prompts, controller, latent=None,  bbox = box, generator=g_cpu)

    for i,path in enumerate(out_path_list):
        view_images(iamges,if_save=True,output_folder=path,save_which=i,img_name=filename)

    print(str(idx+1)+"/"+str(len(file_list)),"done!")

